In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
df=pd.read_csv("root_cause_analysis.csv")

In [3]:
df

,ID,CPU_LOAD,MEMORY_LEAK_LOAD,DELAY,ERROR_1000,ERROR_1001,ERROR_1002,ERROR_1003,ROOT_CAUSE
0,1,0,0,0,0,1,0,1,MEMORY_LEAK
1,2,0,0,0,0,0,0,1,MEMORY_LEAK
2,3,0,1,1,0,0,1,1,MEMORY_LEAK
3,4,0,1,0,1,1,0,1,MEMORY_LEAK
4,5,1,1,0,1,0,1,0,NETWORK_DELAY
...,...,...,...,...,...,...,...,...,...
995,996,0,0,0,0,0,0,1,DATABASE_ISSUE
996,997,0,0,0,1,0,0,0,NETWORK_DELAY
997,998,1,1,1,0,0,0,0,MEMORY_LEAK
998,999,0,1,1,1,1,0,0,NETWORK_DELAY


In [4]:
df["ROOT_CAUSE"].nunique()

3

In [5]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
df["ROOT_CAUSE"] = label_encoder.fit_transform(df["ROOT_CAUSE"]) 
df

,ID,CPU_LOAD,MEMORY_LEAK_LOAD,DELAY,ERROR_1000,ERROR_1001,ERROR_1002,ERROR_1003,ROOT_CAUSE
0,1,0,0,0,0,1,0,1,1
1,2,0,0,0,0,0,0,1,1
2,3,0,1,1,0,0,1,1,1
3,4,0,1,0,1,1,0,1,1
4,5,1,1,0,1,0,1,0,2
...,...,...,...,...,...,...,...,...,...
995,996,0,0,0,0,0,0,1,0
996,997,0,0,0,1,0,0,0,2
997,998,1,1,1,0,0,0,0,1
998,999,0,1,1,1,1,0,0,2


In [6]:
df1=df.to_numpy()
df1

array([[   1,    0,    0, ...,    0,    1,    1],
       [   2,    0,    0, ...,    0,    1,    1],
       [   3,    0,    1, ...,    1,    1,    1],
       ...,
       [ 998,    1,    1, ...,    0,    0,    1],
       [ 999,    0,    1, ...,    0,    0,    2],
       [1000,    1,    0, ...,    1,    0,    0]], dtype=int64)

In [7]:
x=df.drop(columns=["ROOT_CAUSE","ID"])
y=df["ROOT_CAUSE"]

In [8]:
x

,CPU_LOAD,MEMORY_LEAK_LOAD,DELAY,ERROR_1000,ERROR_1001,ERROR_1002,ERROR_1003
0,0,0,0,0,1,0,1
1,0,0,0,0,0,0,1
2,0,1,1,0,0,1,1
3,0,1,0,1,1,0,1
4,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,1
996,0,0,0,1,0,0,0
997,1,1,1,0,0,0,0
998,0,1,1,1,1,0,0


In [9]:
y

0      1
1      1
2      1
3      1
4      2
      ..
995    0
996    2
997    1
998    2
999    0
Name: ROOT_CAUSE, Length: 1000, dtype: int32

In [10]:
from tensorflow import keras
y=keras.utils.to_categorical(y,3)

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2, random_state=42)

In [12]:
X_train.shape

(800, 7)

In [13]:
y_train.shape

(800, 3)

In [14]:
import tensorflow as tf
from tensorflow import keras
nb_class=3
model=tf.keras.models.Sequential()
model.add(keras.layers.Dense(128,input_shape=(7,),name="hidden_layer_1",activation="relu"))
model.add(keras.layers.Dense(128,name="hidden_layer_2", activation="relu"))
model.add(keras.layers.Dense(3,name="hidden_layer_3",activation="softmax"))
model.summary()


c:\Users\deepa\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hidden_layer_1 (Dense)          │ (None, 128)            │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_layer_2 (Dense)          │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_layer_3 (Dense)          │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,923 (70.01 KB)

 Trainable params: 17,923 (70.01 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
#Compile the model with the hyper parameters loss and metrics
model.compile(loss="categorical_crossentropy",metrics=["accuracy"])

In [16]:
#Building The model
BATCH_SIZE=64
EPOCH=20
VERBOSE=1  #Displays progress bar and logs during training
VALIDATION_SPLIT=0.2
model.fit(X_train,y_train,batch_size=BATCH_SIZE,epochs=EPOCH,verbose=VERBOSE,validation_split=VALIDATION_SPLIT)
model.evaluate(X_test,y_test)



Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.5889 - loss: 0.9783 - val_accuracy: 0.7625 - val_loss: 0.7760
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7942 - loss: 0.7234 - val_accuracy: 0.7812 - val_loss: 0.6443
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8266 - loss: 0.5776 - val_accuracy: 0.7875 - val_loss: 0.5779
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8094 - loss: 0.5221 - val_accuracy: 0.7812 - val_loss: 0.5567
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8053 - loss: 0.4973 - val_accuracy: 0.8000 - val_loss: 0.5416
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8297 - loss: 0.4397 - val_accuracy: 0.8125 - val_loss: 0.5385
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8592 - loss: 0.4027 - val_accuracy: 0.7812 - val_loss: 0.5327
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8344 - loss: 0.4448 - val_accuracy: 0.8062 - val_lo

[0.41864749789237976, 0.8450000286102295]

In [17]:
df.columns

Index(['ID', 'CPU_LOAD', 'MEMORY_LEAK_LOAD', 'DELAY', 'ERROR_1000',
       'ERROR_1001', 'ERROR_1002', 'ERROR_1003', 'ROOT_CAUSE'],
      dtype='object')

In [18]:
new_data=[1,0,0,0,1,1,0]
new_data1=[1,0,1,0,1,0,1]
# new_data_array=np.array([new_data,new_data1])

pred=np.argmax(model.predict(np.array([new_data,new_data1])),axis=1)

print(label_encoder.inverse_transform(pred))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
['DATABASE_ISSUE' 'MEMORY_LEAK']
